In [1]:
import keras
import keras.layers as lay
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from src.model import Generator
from src.model import Discriminator

In [2]:
gen = Generator(name='Generator')
dis = Discriminator(name='Discriminator')
# dis.summary()
# gen.summary()

In [52]:
dis.stage.assign(1)
label = dis(tf.random.normal((1,8,8,3))).numpy()
print(label)

[[0.50313294]]
